# 642: Designing a search auto-complete system
Design a search auto-complete system for search engines. The user can enter a sentence (at least one word and end with a special character '#'). For each character except '#', you need to return the first 3 historical hot sentences that are the same as the prefix of the sentence part you have entered. The specific rules are as follows:

The heat of a sentence is defined as the number of times the user enters the exact same sentence. 
The first 3 popular sentences returned should be sorted by popularity (the first one is the hottest). If the heat of several sentences is the same, you need to use the ascii code order (the smaller one is displayed first). 
If there are less than 3 popular sentences, return as much as possible. 
When the input is a special character, it means the sentence ends, in which case you need to return an empty list. 
Your job is to implement the following features:

Constructor:

1. AutocompleteSystem(String[] sentence, int[] times): This is the constructor. The input is historical data. A sentence is an array of strings consisting of previously entered sentences. Times is the corresponding number of times a sentence is entered. Your system should record this historical data.

Now the user wants to enter a new sentence. The following function will provide the next character of the user type:

2. ListInput(char c): Input c is the next character entered by the user. Characters can only be lowercase letters ("a" to "z"), spaces (""), or special characters ("#"). In addition, the sentences entered earlier should be recorded in the system. The output will be the first 3 historical hot sentences with the same prefix as the sentence part already entered.

## Topic analysis
Design a search auto-completion system that needs to include the following two methods:

Construction method:
AutocompleteSystem(String[] sentences, int[] times): Enter sentence sentences and their occurrences times

Input method:
ListInput(char c): The input character c can be 26 lowercase English letters, or it can be a space, ending with '#'. Returns the highest frequency of up to 3 sentences in the input character prefix. When the frequencies are equal, they are arranged in lexicographic order.

Thought analysis:
Core point: Trie (dictionary tree)

Use the dictionary tree to record all the set of sentences that have appeared, and use the dictionary to save the number of occurrences of each sentence.

Problem solving
The requirement of the title is that the completed sentences are arranged according to the frequency that appeared before, and the high frequencies appear at the top. If the frequencies are the same, they are displayed in alphabetical order.

The frequency requirement is easy to think of heaps, priority queues, trees, maps and other knowledge points. This involves the dictionary and the tree, which must be solved using the dictionary tree.

So first construct Trie's trieNode structure and insert method, construct the trieNode class, and then construct a tree's root node.

Since we have to enter one character at a time, we can use a private Node:curNode to track the current node.

CurNode is initialized to root. We need to set it to root each time we enter a sentence, that is, if the input character is '#'.

Also need a string type stn to represent the current search sentence.

Each time you enter a character, first check if the end tag "#", if so, add the current sentence to the trie tree, reset the relevant variables, and return an empty array.

If not, check if the child corresponding to the current TrieNode contains the corresponding node of c. If not, set curNode to NULL and return an empty array.

If it exists, update the curNode to the node corresponding to c, and perform dfs on the curNode.

For dfs, we first check if the current sentence is a complete sentence. If it is, add the sentence to the priority_queue at the same time as its number, and then dfs the child nodes that may exist in the child.

After the dfs is completed, only the first three need to be taken out. It should be noted that the possible result may be less than three, so to add more conditional statements in the while to detect that q is empty.

Finally, all the elements in q will be popped up.

In [ ]:
class TrieNode{
  public:
    string str;
    int cnt;
    unordered_map<char, TrieNode*> child;
    TrieNode(): str(""), cnt(0){};
};

struct cmp{
    bool operator() (const pair<string, int> &p1, const pair<string, int> &p2){
        return p1.second < p2.second || (p1.second == p2.second && p1.first > p2.first);
    }
};

class AutocompleteSystem {
public:
    AutocompleteSystem(vector<string> sentences, vector<int> times) {
        root = new TrieNode();
        for(int i = 0; i < sentences.size(); i++){
            insert(sentences[i], times[i]);
        }
        curNode = root;
        stn = "";
    }
    
    vector<string> input(char c) {
        if(c == '#'){
            insert(stn, 1);
            stn.clear();
            curNode = root;
            return {};
        }
        stn.push_back(c);
        if(curNode && curNode->child.count(c)){
            curNode = curNode->child[c];
        }else{
            curNode = NULL;
            return {};
        }
        
        dfs(curNode);
        
        vector<string> ret;
        int n = 3;
        while(n > 0 && !q.empty()){
            ret.push_back(q.top().first);
            q.pop();
            n--;
        }
        while(!q.empty()) q.pop();
        
        return ret;
    }
    
    void dfs(TrieNode* n){
        if(n->str != ""){
            q.push({n->str, n->cnt});
        }
        for(auto p : n->child){
            dfs(p.second);
        }
    }
    
    void insert(string s, int cnt){
        TrieNode* cur = root;
        for(auto c : s){
            if(cur->child.count(c) == 0){
                cur->child[c] = new TrieNode();
            }
            cur = cur->child[c];
        }
        cur->str = s;
        cur->cnt += cnt;
    }
    
private:
    TrieNode *root, *curNode;
    string stn;
    priority_queue<pair<string,int>, vector<pair<string, int>>, cmp > q;
    
};
